In [ ]:
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info
from ipywidgets import Dropdown

In [ ]:
# モデルの選択（2B モデルか 7B モデルかを選択）
model_options = ["Qwen/Qwen2-VL-2B-Instruct", "Qwen/Qwen2-VL-7B-Instruct"]
model_dropdown = Dropdown(options=model_options, value=model_options[0], description='Model:')

# ドロップダウンを表示
display(model_dropdown)

def get_model_id(change):
    global model_id
    model_id = change.new

# 値が変更されたときにget_model_id関数を呼び出す
model_dropdown.observe(get_model_id, names='value')

model_id = model_dropdown.value


In [ ]:
# モデルをロード
print(f"loading {model_id}...")
model = Qwen2VLForConditionalGeneration.from_pretrained(
    model_id, torch_dtype="auto", device_map="auto"
)

In [ ]:
# プロセッサの初期化
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-2B-Instruct")

In [ ]:
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": "chat_with_ai.jpg",
            },
            {"type": "text", "text": "Describe this image."},
        ],
    }
]

In [ ]:
# プロンプトの準備
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
print(text)


In [ ]:
# イメージの準備
image_inputs, video_inputs = process_vision_info(messages)

In [ ]:
# モデル入力の準備
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")

In [ ]:
# 推論：出力の生成
generated_ids = model.generate(**inputs, max_new_tokens=128)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text)